Messdaten im table - Format

In [15]:
import pandas as pd
import numpy as np
runderstab = pd.read_excel(r"tables/runderstabeinseitig.xlsx")
eckigerstab = pd.read_excel(r"tables/eckigerstabeinseitig.xlsx")
#print(runderstab.to_latex(index = False, column_format= "c c c", na_rep = "-", float_format="%.3f"))
#print(eckigerstab.to_latex(index = False, column_format= "c c c", na_rep = "-", float_format="%.3f"))